In [109]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [110]:
from machine import Pin, I2C

i2c = I2C(scl=Pin(15), sda=Pin(4), freq=450000)
print(list(map(hex, i2c.scan())))
i2c.writeto_mem(0x29, 0x80, b'\x03')
print("chipid", hex(i2c.readfrom_mem(0x29, 0x92, 1)[0]))

i2c.writeto_mem(0x29, 0x8F, b'\x00')  # gain of 1x
i2c.writeto_mem(0x29, 0x81, b'\x00')  # integration time 700ms

i2c.writeto_mem(0x29, 0x8F, b'\x01')  # gain of 2x
i2c.writeto_mem(0x29, 0x81, b'\xC0')  # integration time 154ms

i2c.writeto_mem(0x29, 0x8F, b'\x02')  # gain of 4x
i2c.writeto_mem(0x29, 0x81, b'\xEB')  # integration time 50ms


['0x29']
chipid 0x44


In [111]:
import struct, time
t0 = time.ticks_ms()
for i in range(20):
    print(time.ticks_ms() - t0, struct.unpack("<HHHH", i2c.readfrom_mem(0x29, 0x94, 8)))
    time.sleep(1)

0 (334, 98, 116, 102)
1002 (387, 110, 139, 129)
2002 (386, 109, 139, 129)
3003 (385, 109, 139, 129)
4004 (385, 109, 138, 129)
5005 (385, 109, 138, 129)
6006 (385, 109, 139, 129)
7006 (386, 109, 139, 129)
8007 (384, 109, 138, 129)
9008 (386, 109, 139, 129)
.10009 (386, 109, 139, 129)
11009 (385, 109, 138, 129)
12010 (384, 109, 138, 128)
13011 (326, 95, 114, 100)
14012 (331, 97, 115, 101)
15012 (331, 97, 115, 101)
16013 (331, 97, 115, 101)
17014 (329, 96, 114, 101)
18015 (328, 96, 114, 100)
19015 (329, 96, 114, 101)
.

In [ ]:
# As an async project to send colours to MQTT

In [158]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [156]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
boardname    esp32colours

Sent 5 lines (115 bytes) to config.txt.


In [ ]:
#%sendtofile main.py

import time, struct
from machine import Pin, I2C
from mqtt_as import config, MQTTClient
import uasyncio as asyncio

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
from machine import Pin, I2C
i2c = I2C(scl=Pin(15), sda=Pin(4), freq=45000)
print(list(map(hex, i2c.scan())))

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
boardname = fconfig["boardname"].encode()    

topiccrgb = boardname+b'/crgb'
client = None

async def TCS34725task():
    i2c.writeto_mem(0x29, 0x80, b'\x03')
    print("chipid", hex(i2c.readfrom_mem(0x29, 0x92, 1)[0]))

    i2c.writeto_mem(0x29, 0x8F, b'\x00')  # gain of 1x
    i2c.writeto_mem(0x29, 0x81, b'\x00')  # integration time 700ms

    i2c.writeto_mem(0x29, 0x8F, b'\x01')  # gain of 2x
    i2c.writeto_mem(0x29, 0x81, b'\xC0')  # integration time 154ms

    i2c.writeto_mem(0x29, 0x8F, b'\x02')  # gain of 4x
    i2c.writeto_mem(0x29, 0x81, b'\xEB')  # integration time 50ms

    i = 0
    while True:
        i += 1
        c,r,g,b = struct.unpack("<HHHH", i2c.readfrom_mem(0x29, 0x94, 8))
        v = "%d %d %d %d"%(c,r,g,b)
        print(v)
        if client and client.isconnected():
            await client.publish(topiccrgb, v)
            pinled.value(i%2)
        else:
            pinled.value((i//20)%2)
        await asyncio.sleep_ms(100)

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+'/ip', ipnumber, retain=True)
    
async def mqtttask(bflip=False):
    await asyncio.sleep_ms(5000)
    if bflip and "wifialt" in fconfig:
        assid, awifi_pw, aserver = fconfig["wifialt"].split(",")
        if client._ssid == assid:
            assid, awifi_pw, aserver = fconfig["wifiname"], fconfig["wifipassword"], fconfig["mqttbroker"]
        client._ssid, client._wifi_pw, client.server = assid, awifi_pw, aserver
    try:
        print("connecting to:", client._ssid)
        await client.connect()
        return
    except OSError as e:
        print("client connect", [e])
    aloop.create_task(mqtttask(True))

print("hi there",1)
time.sleep(1)
config['connect_coro'] = onconnecttask
client = MQTTClient(config)
    
print("hi there")
aloop = asyncio.get_event_loop()
aloop.create_task(TCS34725task())
aloop.create_task(mqtttask())
aloop.run_forever()



['0x29']
hi there 1
hi there
chipid 0x44
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
7659 1966 2707 2532
631 162 223 209
630 162 223 208
633 162 224 209
632 162 223 209
630 162 223 208
632 162 224 209
633 162 224 209
630 162 223 208
631 162 223 209
633 162 224 209
631 162 223 208
631 162 223 209
633 162 224 209
632 162 223 209
630 162 223 208
633 162 224 209
632 162 223 209
630 162 223 208
632 162 223 209
633 162 224 209
630 162 223 208
632 162 223 209
633 162 224 209
631 162 223 209
631 162 223 209
631 162 223 209
633 162 224 209
632 162 223 209
630 162 223 208
633 162 224 209
633 162 224 209
630 162 223 208
632 162 223 209
633 162 224 209
630 162 223 208
631 162 223 209
633 162 224 209
631 162 223 208
631 162 223 208
633 162 224 209
connecting to: DoESLiverpool
632 162 223 209
630 162 223 208
632 162 223 209
632 162 223 209
630 162 223 208
632 162 223 209
631 162 223 209
628 161 222 208
629 161 222 208
631 162 223 208
624 161 221 206
605 157 213 199
584 154 205 190
499 133 172 16

In [152]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [154]:
import os
os.rename("dmain.py", "main.py")

In [125]:
import network
w = network.WLAN()

In [126]:
w.active(1)


Brownout detector was triggered

ets Jun  8 2016 00:22:57

rst:0xc (SW_CPU_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4928
ho 0 tail 12 room 4
load:0x40078000,len:9404
load:0x40080400,len:6228
entry 0x400806ec
MicroPython v1.11-240-g519746cae on 2019-08-26; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' 

In [62]:
x = TCS34725(i2c)

In [144]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [ ]:
#%fetchfile --load main.py

# full main.py
# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
#i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
import machine, time, ubinascii

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

from machine import Pin, I2C
import uasyncio as asyncio
from fmqttqueue import FMQTTQueue

fconfig = dict(x.split()  for x in open("config.txt"))
pled = Pin(int(fconfig["pinled"]), Pin.OUT)

# change baud for some 
for n in range(5):
    devices = IdentifyI2CDevice(i2c)
    print("devices", devices)
    if devices:
        break
    time.sleep(0.3)

if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    # gps is 9600, bluefly is 57600  
    devices = IdentifyUARTDevice(uart)
    
if not devices or fconfig["boardname"] == "esp32silver2":
    # Pin 13=D7 on esp8266
    # dallas wiring: flat face towards you, 
    # left is ground, right is live, 
    # middle is signal with a 4.7k pullup resistor onto the live
    # on string white=G, green=Live, red=signal
    pd = 18 if fconfig["boardname"] == "esp32silver2" else 13
    import onewire, ds18x20
    pindallas = machine.Pin(pd, Pin.OUT, Pin.PULL_UP)
    dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
    dallasscanned = dallasobj.scan()
    if dallasscanned:
        sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
        dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
        devices.append("DS18B20 n=%d"%len(dallasscanned))
    print("dallas", devices)
        
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
devnames = [ ]
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    devnames.append(devname)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()
    time.sleep(1)

async def plotdevices(fqm):
    topic = fconfig["boardname"]+"/"+devname
    if devname == "VL53L0X":
        from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
        VL53L0Xinit(i2c)
        while True:
            d = VL53L0Xdist()
            fbuff.fill(0)
            fbuff.text("VL53L0X", 0, 0, 1)
            fbuff.text("dist(mm):", 8, 20, 1)
            fatntext("%d" % d, 40, 36)
            oledshow()
            fqm.fput(topic, "%d"%d)
            await asyncio.sleep_ms(50)
        
    if devname == "VL6180":
        from VL6180_funcs import VL6180init, distmm
        VL6180init(i2c)
        while True:
            d = distmm()
            fbuff.fill(0)
            fbuff.text("VL6180", 0, 0, 1)
            fbuff.text("dist(mm):", 8, 20, 1)
            fatntext("%d" % d, 40, 36)
            oledshow()
            fqm.fput(topic, "%d" % d)
            await asyncio.sleep_ms(50)

    if devname == "BME280" or devname == "BME180":
        from BME280_funcs import bme280init, readBME280
        bme280init(i2c)
        topictemp = topic+"/temp"
        topichumid = topic+"/humid"
        topicpressure = topic+"/pressure"
        n = 0
        while True:
            temp, pressure, humid = readBME280()
            fbuff.fill(0)
            fbuff.text("BME280", 0, 0, 1)
            fbuff.text("tmp:", 0, 16, 1)
            fatntext("%.2f"%(temp), 32, 10)
            fbuff.text("prs:", 0, 36, 1)
            fatntext("%.2f"%(pressure), 32, 30)
            fbuff.text("hum:", 0, 54, 1)
            fatntext("%.2f"%(humid), 32, 50)
            oledshow()
            if (n % 5) == 0:
                fqm.fput(topichumid, "%.2f"%humid)
                fqm.fput(topictemp, "%.2f"%temp)
                fqm.fput(topicpressure, "%.2f"%pressure)
            n += 1
            await asyncio.sleep_ms(110)

    if devname == "SDOF":
        from SDOF_funcs import SetupAccGyrMag, readvectorsensor
        SetupAccGyrMag(i2c)
        topicaccx = topic+"/acc/x"
        topicaccy = topic+"/acc/y"
        topicaccz = topic+"/acc/z"
        n = 0
        while True:
            fbuff.fill(0)
            ax, ay, az = readvectorsensor("a")
            fbuff.text("Acc:", 0, 0, 1)
            fbuff.text("%d"%ax, 8, 8, 1)
            fbuff.text("%d"%ay, 8, 16, 1)
            fbuff.text("%d"%az, 8, 24, 1)
            x, y, z = readvectorsensor("g")
            fbuff.text("gyr:", 64, 0, 1)
            fbuff.text("%d"%x, 72, 8, 1)
            fbuff.text("%d"%y, 72, 16, 1)
            fbuff.text("%d"%z, 72, 24, 1)
            x, y, z = readvectorsensor("c")
            fbuff.text("mag:", 40, 32, 1)
            fbuff.text("%d"%x, 48, 40, 1)
            fbuff.text("%d"%y, 48, 48, 1)
            fbuff.text("%d"%z, 48, 56, 1)
            oledshow()
            
            if (n % 5) == 0:
                fqm.fput(topicaccx, "%d"%ax)
                fqm.fput(topicaccy, "%d"%ay)
                fqm.fput(topicaccz, "%d"%az)
            n += 1
            await asyncio.sleep_ms(100)
            

    if devname == "MLX90614":
        from MLX90614_funcs import mlx90614temps
        from sevensegmentdisplay import encodeledstring # wired to heltecwhite
        i2c.init(scl=Pin(15), sda=Pin(4), freq=100000) # can't handle higher frequencies
        topicambient = topic+"/temp/ambient"
        topicir = topic+"/temp/ir"
        n = 0
        while True:
            tempAmbient, tempIR = mlx90614temps(i2c)
            if tempAmbient is not None:
                encodeledstring("%.2f c"% tempIR)
                if (n % 5) == 0 and tempIR > -10:
                    fqm.fput(topicambient, "%.2f"%tempAmbient)
                    fqm.fput(topicir, "%.2f"%tempIR)
                n += 1
            if fqm.rqueue:
                topic, msg = fqm.rqueue.popleft()
                encodeledstring(msg)
                await asyncio.sleep_ms(800)
            await asyncio.sleep_ms(200)
            
    if devname == "LP55231":
        fqm.addsubscription("LP55231/#")
        while True:
            if fqm.rqueue:
                topic, msg = fqm.rqueue.popleft()
                print((topic, msg))
                fbuff.fill(0)
                fbuff.text("LP55231", 0, 0, 1)
                fbuff.text("topic:", 0, 8, 1)
                fbuff.text(topic.decode(), 0, 16, 1)
                fbuff.text("msg:", 0, 53, 1)
                fatntext(msg.decode(), 50, 46)
                oledshow()
                try:
                    pos, col = topic.decode().split("/")[1:]
                    intensity = int(msg)
                    if col == "red":
                        bpos = 0x1C + int(pos)
                    else:
                        bpos = (0x16 if col == "green" else 0x17) + int(pos)*2
                    i2c.writeto(0x32, bytes((bpos, intensity)))
                except ValueError as e:
                    print(e)
            else:
                await asyncio.sleep_ms(5)

async def CDM7160_task():
    from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
    from CDM7160_funcs import readCO2Filtered, setupCO2filters

    topicco2 = fconfig["boardname"]+"/CDM7160"
    topicco2smooth = fconfig["boardname"]+"/CDM7160smooth"
    scrollinit()
    setupCO2filters(i2c)
    
    while True:
        pled.value(1)
        await asyncio.sleep_ms(50)
        pled.value(0)
        n, co2ppm, filtco2ppm = readCO2Filtered()
        print(n, co2ppm, filtco2ppm)
        if co2ppm != 0.0:
            fbuff.fill(0)
            addscrollgraph(co2ppm, time.ticks_ms())
            plotscrollgraph(fbuff)
            fbuff.text("CO2 %d ppm" % co2ppm, 0, 0)
            oledshow()
            fqm.fput(topicco2, "%d" % co2ppm)
            fqm.fput(topicco2smooth, "%d" % filtco2ppm)
            await asyncio.sleep_ms(3000)
        else:
            fbuff.fill(0)
            fbuff.text("figaroCO2  BAD", 0, 0)
            fbuff.text("n=%d"%n, 16, 8)
            fbuff.text("%d"%(-filtco2ppm), 8, 16)
            doublepixels()
            oledshow()
            await asyncio.sleep_ms(1000)
                
                
async def DS18B20_task():
    topicds = [ fconfig["boardname"]+"/DS18B20/"+str(i)  for i in range(len(dallasscanned)) ]
    while True:
        dallasobj.convert_temp()
        await asyncio.sleep_ms(800)
        fbuff.fill(0)
        fbuff.text("DS18B20 temps", 0, 0, 1)
        for i, d in enumerate(dallasscanned):
            try:
                t = dallasobj.read_temp(d)
            except Exception:
                t = -100
            if len(dallasscanned) != 1:
                fbuff.text("%.2f"%t, (i%2)*64, (i//2+2)*8, 1)
            else:
                fatntext("%.2f"%t, 8, 20)
            if t != -100:
                fqm.fput(topicds[i], "%.2f"%t)
        if fqm.rqueue:
            topic, msg = fqm.rqueue.popleft()
            fbuff.text(msg.decode(), 0, 56, 1)
        oledshow()

async def TMP102_task():
    topic = fconfig["boardname"]+"/TMP102"
    n = 0
    while True:
        try:
            r = i2c.readfrom_mem(0x48, 0x00, 2)
            t = r[0] + r[1]/256
            fbuff.fill(0)
            fbuff.text("TMP102", 0, 0, 1)
            fbuff.text("temp:", 0, 23, 1)
            fatntext("%.03f"%t, 30, 40)
            if fqm.rqueue:
                fbuff.text(fqm.rqueue.popleft()[1].decode(), 0, 56, 1)
            oledshow()
            if (n % 5) == 0:
                fqm.fput(topic, "%.03f"%t)
            n += 1
        except OSError as e:
            print(e, "TMP102_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(100)
            
async def SHT31D_task():
    topichumid = fconfig["boardname"]+"/SHT31D/humid"
    topictemp = fconfig["boardname"]+"/SHT31D/temp"
    n = 0
    from SHT31D_funcs import SHT31Dsetup, readSHT31D
    SHT31Dsetup(i2c)
    while True:
        try:
            tmp, humid = readSHT31D()
            fbuff.fill(0)
            fbuff.text("SHT31-D", 0, 0, 1)
            fbuff.text("temperature:", 10, 10, 1)
            fatntext("%.2f"%tmp, 20, 22)
            fbuff.text("humidity:", 10, 40, 1)
            fatntext("%.2f"%humid, 20, 50)
            oledshow()
            if (n % 5) == 0:
                fqm.fput(topichumid, "%.2f"%humid)
                fqm.fput(topictemp, "%.2f"%tmp)
            n += 1
        except OSError as e:
            print(e, "SHT31D_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(100)
            
async def TSL561_task():
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    topicbr = fconfig["boardname"]+"/TSL561/broad"
    topicir = fconfig["boardname"]+"/TSL561/ir"
    setupTSL561(i2c)
    setregtimings(True, 1)
    n = 0
    while True:
        try:
            br, ir = luminosity()
            fbuff.fill(0)
            fbuff.text("TSL561", 0, 0, 1)
            fbuff.text("broad:", 0, 23, 1)
            fatntext("%d"%br, 50, 16)
            fbuff.text("ir:", 0, 53, 1)
            fatntext("%d"%ir, 50, 46)
            oledshow()
            if (n % 10) == 0:
                fqm.fput(topicir, "%d"%ir)
                fqm.fput(topicbr, "%d"%br)
            n += 1
        except OSError as e:
            print(e, "TSL561_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(50)
            
async def MS5611_task():
    from MS5611_funcs import ms5611setup, readMS5611
    from OLED_grapher import scrollinit, addscrollgraph, plotscrollgraph
    scrollinit()
    topicbaro = fconfig["boardname"]+"/MS5611/baro"
    ms5611setup(i2c)
    n = 0
    while True:
        try:
            temp, pr = readMS5611()
            #fbuff.fill(0)
            #fbuff.text("TSL561", 0, 0, 1)
            #fbuff.text("temp:", 0, 23, 1)
            #fatntext("%f"%temp, 50, 16)
            #fbuff.text("prs:", 0, 35, 1)
            #fatntext("%f"%pr, 0, 46)
            addscrollgraph(pr, time.ticks_ms())
            plotscrollgraph(fbuff)
            oledshow()
            if (n % 10) == 0:
                fqm.fput(topicbaro, "%f"%pr)
            n += 1
        except OSError as e:
            print(e, "MS5611_task")
            await asyncio.sleep_ms(1000)
        await asyncio.sleep_ms(10)

async def BNO055_task():
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    binit = False
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            if not binit:
                InitBNO055(uart)
                binit = True
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        await asyncio.sleep_ms(100)
        
async def SI7021_task():
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    topichumid = fconfig["boardname"]+"/SI7021/humid"
    topictemp = fconfig["boardname"]+"/SI7021/temp"
    print(topictemp)
    n = 0
    while True:
        h, t = SI7021humiditytemp()
        print(h, n)
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        if fqm.rqueue:
            fbuff.text(fqm.rqueue.popleft()[1].decode(), 0, 32)
        oledshow()
        if (n % 4) == 0:
            fqm.fput(topichumid, "%.1f"%h)
            fqm.fput(topictemp, "%.1f"%t)
        n += 1
        await asyncio.sleep_ms(250)
        
async def MPU6050_task():
    from MPU6050_funcs import setupDMP, calibgyros, calibacc
    from MPU6050_funcs import getFIFOCount, setDMPEnabled, getIntStatus, readfifoBuffer, ConvertQuatToEuler
    import ustruct
    
    setupDMP(i2c, "MPU6050_dmpMemoryv6.hex")
    calibgyros((80, -14, 30))
    calibacc((582, 951, 1692))

    topicorient = fconfig["boardname"]+"/MPU6050/orient"
    print(topicorient)
    setDMPEnabled(True)
    n = 0
    while True:
        fifoBuffer = None
        fifoCount = getFIFOCount()
        if fifoCount >= 900:
            setDMPEnabled(True)  # reset the fifo buffer
        while getFIFOCount() >= 28:
            fifoBuffer = readfifoBuffer(28)
        if fifoBuffer is not None:
            n += 1
            q0, l, q1, l, q2, l, q3, l = ustruct.unpack(">hhhhhhhh", fifoBuffer)
            pitch, roll, orient = ConvertQuatToEuler(q0, q1, q2, q3)
            fqm.fput(topicorient, "%.4f %.4f %.4f"%(pitch, roll, orient))
            await asyncio.sleep_ms(1)
        else:
            await asyncio.sleep_ms(5)
        
async def IPnumber_task(fqm):
    topicip = fconfig["boardname"]+"/ip"
    topicmac = fconfig["boardname"]+"/mac"
    while True:
        if fqm.client and fqm.client.isconnected():
            await fqm.client.publish(topicip, fqm.client._sta_if.ifconfig()[0], retain=True, qos=1)
            await fqm.client.publish(topicmac, ubinascii.hexlify(fqm.client._sta_if.config('mac'),':').decode().upper(), retain=True, qos=1)
            await asyncio.sleep_ms(60000)
        else:
            await asyncio.sleep_ms(1000)

        
# setup and operation of the async loop
fqm = FMQTTQueue(pled)
fqm.setupmqttas(fconfig["wifiname"], fconfig.get("wifipassword", ""), fconfig["mqttbroker"])
fqm.addsubscription(fconfig["boardname"]+"/message")

aloop = asyncio.get_event_loop()
aloop.create_task(fqm.fstreamtask())
aloop.create_task(IPnumber_task(fqm))

if "TMP102" in devnames:
    aloop.create_task(TMP102_task())
if "SHT31D" in devnames:
    aloop.create_task(SHT31D_task())
if "TSL561" in devnames:
    aloop.create_task(TSL561_task())
if "MS5611" in devnames:
    aloop.create_task(MS5611_task())
if "CDM7160" in devnames:
    aloop.create_task(CDM7160_task())
if "DS18B20" in devnames:
    aloop.create_task(DS18B20_task())
if "BNO055" in devnames:
    aloop.create_task(BNO055_task())
print(devnames)
if "SI7021" in devnames:
    aloop.create_task(SI7021_task())
if "MPU6050" in devnames:
    aloop.create_task(MPU6050_task())

aloop.run_until_complete(plotdevices(fqm))
aloop.run_forever()
